In [36]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
import sys
import os
import json
import torch
import yaml
import numpy as np
import cv2
import copy
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image

In [38]:
DATA_PATH = "/workspace/plane-nerf/data/jackal_floor_training_data_1"

#Delete yolo
if os.path.exists(DATA_PATH + "/yolo"):
    os.system("rm -rf " + DATA_PATH + "/yolo")

if not os.path.exists(DATA_PATH + "/yolo"):
    os.makedirs(DATA_PATH + "/yolo")

if not os.path.exists(DATA_PATH + "/yolo/train/images"):
    os.makedirs(DATA_PATH + "/yolo/train/images")

if not os.path.exists(DATA_PATH + "/yolo/train/labels"):
    os.makedirs(DATA_PATH + "/yolo/train/labels")

if not os.path.exists(DATA_PATH + "/yolo/valid/images"):
    os.makedirs(DATA_PATH + "/yolo/valid/images")

if not os.path.exists(DATA_PATH + "/yolo/valid/labels"):
    os.makedirs(DATA_PATH + "/yolo/valid/labels")

with open(DATA_PATH + "/transforms.json") as f:
    transforms = json.load(f)

In [39]:
#Create data.yaml file
names_dictionary = {
    0 : "jackal"
}

data = {
    "path": DATA_PATH + "/yolo/",
    "train": "train/images",
    "val": "valid/images",
    "nc": 1,
    "names": ["jackal"]
}

with open(DATA_PATH + "/yolo/data.yaml", 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)
    

In [40]:
training_ratio = 0.8
n = len(transforms["frames"])
frames_index = list(range(n))
np.random.shuffle(frames_index)

for i in range(int(n*training_ratio)):
    img = Image.open(DATA_PATH + "/images/" + str(frames_index[i]) + ".png")
    img.save(DATA_PATH + "/yolo/train/images/" + str(frames_index[i]) + ".png")

    mask = Image.open(DATA_PATH + "/masks/" + str(frames_index[i]) + ".png")
    #Find contours of mask
    mask = np.array(mask)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #Create yolo label file for segmentation
    with open(DATA_PATH + "/yolo/train/labels/" + str(frames_index[i]) + ".txt", "w") as f:
        contour_text = "0 "
        for contour in contours:
            for point in contour:
                contour_text += str(point[0][0]) + " " + str(point[0][1]) + " "
        
        f.write(contour_text)
    
for i in range(int(n*training_ratio), n):
    img = Image.open(DATA_PATH + "/images/" + str(frames_index[i]) + ".png")
    img.save(DATA_PATH + "/yolo/valid/images/" + str(frames_index[i]) + ".png")

    mask = Image.open(DATA_PATH + "/masks/" + str(frames_index[i]) + ".png")
    #Find contours of mask
    mask = np.array(mask)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #Create yolo label file for segmentation
    with open(DATA_PATH + "/yolo/valid/labels/" + str(frames_index[i]) + ".txt", "w") as f:
        contour_text = "0 "
        for contour in contours:
            for point in contour:
                contour_text += str(point[0][0]) + " " + str(point[0][1]) + " "
        
        f.write(contour_text)